In [1]:
#general imports
import pickle

#mesh processing
from skimage import measure
from stl import mesh

In [2]:
#Test notebook for tracking rigid body  marker consisting of multiple fiducials within a CT scan volume

#3D data processing
import open3d as o3d
import ct_tracking_library # our helper functions
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
import os
#our processing code
#from ct_tracking_library.ct_processing_functions import *

#used for GUI
%matplotlib notebook
from ct_tracking_library import gui
from ct_tracking_library.ct_tracking_functions import *

In [3]:
def display_Dicom(file1,file2):
    assert isinstance(file1,str)
    assert isinstance(file1,str)
    img1 = sitk.ReadImage(file1)
    #img2 = sitk.ReadImage(file2)
    img2 = sitk.GetImageFromArray(file2)
    print_info(img1)
    print_info(img2)
    # Obtain foreground masks for the two images using Otsu thresholding, we use these later on.
    msk1 = sitk.OtsuThreshold(img1,0,1)
    msk2 = sitk.OtsuThreshold(img2,0,1)

    gui.MultiImageDisplay(image_list = [img1, img2],
                          title_list = ['Original', 'Estimated'],
                          figure_size=(9,3));

In [4]:
def print_info(selected_image):
    print('origin: ' + str(selected_image.GetOrigin()))
    print('size: ' + str(selected_image.GetSize()))
    print('spacing: ' + str(selected_image.GetSpacing()))
    print('direction: ' + str(selected_image.GetDirection()))
    print('pixel type: ' + str(selected_image.GetPixelIDTypeAsString()))
    print('number of pixel components: ' + str(selected_image.GetNumberOfComponentsPerPixel()))

In [ ]:
display_Dicom("temp_mesh.mha",nda)

In [ ]:
#choose target marker
target_marker = 'marker2'

#convert scan to mesh for finding markers via thresholding
# !!! comment the below code to change the way file is loaded !!!
#convert_scan_to_mesh(scan_file='./test_data/bb_marker3.pkl')
convert_scan_to_mesh_mha(scan_file='./test_data/biopsy 2')
#find candidate fiducials within the scanner returning clustered centroids that could be rigid body markers
marker, marker_centroid_coordinates, pcd_selected, mesh, coordinate_frame, good_centroid_clusters = find_candidate_centroids(target_marker = target_marker)

#display candidate fiducials
o3d.visualization.draw_geometries([pcd_selected, mesh])
print('marker centroid coordinates from design: {}'.format(marker))
print('found centroid coordinates: {}'.format(marker_centroid_coordinates))

#find best transform and marker from candidate fiducials
final_R, final_t, permuted_centroids, min_error = find_best_transform_from_candidate_marker_clusters(marker, good_centroid_clusters)

In [ ]:
#display tracked  markers and coordinate frames
marker = np.load('./test_data/'+target_marker+'.npy')
marker_3d_base, marker_3d_transformed, err = visualize_tracked_marker(marker, final_R, final_t, permuted_centroids)
o3d.visualization.draw_geometries(marker_3d_base+marker_3d_transformed)

#tracking error from final transform
np.set_printoptions(2)
print(err)
print(np.linalg.norm(err))

#visualize everything at once
visualization_list = [pcd_selected, mesh]+marker_3d_base+marker_3d_transformed
o3d.visualization.draw_geometries(visualization_list)